# City Sustainability - CNET Model Workflow

## Imports

In [ ]:
# Imports

import tensorflow as tf
from tensorflow import keras
import matplotlib.pyplot as plt
from tensorflow.keras import layers
from city_sustainability.models import unet_model
from city_sustainability.quality import life_quality
from tensorflow.keras.callbacks import EarlyStopping
from city_sustainability.loading_data import split_data
from city_sustainability.preprocessing import image_resize
from city_sustainability.loading_data import loading_paths
from city_sustainability.loading_data import image_and_label_arrays

## Load and process images

In [ ]:
# Paths for input data and directory containing the train.txt, test.txt, and val.txt files

data_path = '../../raw_data/OpenEarthMap_wo_xBD'
split_files_path = '../../raw_data/OpenEarthMap_wo_xBD/'
(X_train_paths,
 y_train_paths,
 X_val_paths,
 y_val_paths,
 X_test_paths,
 y_test_paths) = split_data (data_path, split_files_path)

print('X_train_paths =', len(X_train_paths))
print('y_train_paths =', len(y_train_paths))
print('X_test_paths =', len(X_test_paths))
print('y_test_paths =', len(y_test_paths))
print('X_val_paths =', len(X_val_paths))
print('y_val_paths =', len(y_val_paths))

In [ ]:
# Load images

X_train, y_train = image_and_label_arrays(X_train_paths, y_train_paths, sampling_ratio=1.0)
# X_test, y_test = image_and_label_arrays(X_train_paths, y_train_paths, sampling_ratio=1.0)
X_val, y_val = image_and_label_arrays(X_val_paths, y_val_paths, sampling_ratio=1.0)

In [ ]:
# Scale X and y

X_train_scaled = X_train / 255
# X_test_scaled = X_test / 255
X_val_scaled = X_val / 255

print(X_train_scaled.shape)
# print(X_test_scaled.shape)
print(X_val_scaled.shape)

## CNET Model - VGG16

In [ ]:
#Load a Trained Model

# path = '../../model_training/mallik/model_3001.h5'
# model = load_model(path, custom_objects={'compute_iou': compute_iou})

In [ ]:
# Create the model

model = build_vgg16_model(input_shape=(256, 256, 3), num_classes = 9)
# When running from .py
# model_full = unet_model.build_vgg16_model_full(input_shape=(400, 400, 3), num_classes = 9)

In [ ]:
# Compile the model

unet_model.compile_model(model)

In [ ]:
# Model Architecture

model.summary()

In [ ]:
# Model Training and Validation

unet_model.train_model(model, X_train_scaled, y_train, epochs = 50, batch_size=64, validation_data = (X_val_scaled, y_val))

In [ ]:
#Save the Trained Model
model.save('../../model_training/marcel')

In [ ]:
# Model Test (prediction)

y_pred = model.predict(X_val_scaled)

In [ ]:
# Verify prediction shape

y_pred.shape

## Original images vs predicted

Dont forget to do CTRL+A and CTRL+P to print the images in .pdf

In [ ]:
# Create a Table DataFrame from the color dictionary

class_colors = {
    "Other": '#FFFF00',
    "Bareland": '#800000',
    "Rangeland": '#00FF24',
    "Developed space": '#949494',
    "Road": '#FFFFFF',
    "Tree": '#226126',
    "Water": '#0045FF',
    "Agriculture land": '#4BB549',
    "Building": '#DE1F07'}

df = pd.DataFrame(list(class_colors.items()), columns=['Class', 'Color'])

def color_cells(val):
    color = val 
    return 'background-color: %s' % color

class_color = df.style.applymap(color_cells, subset=['Color'])
class_color

In [ ]:
#Plot images to compare: Satellite Original, Label Original and Label Prediction.

# Create Numpy
y_pred_np = np.argmax(y_pred, axis=-1)
y_val_np = np.argmax(y_val, axis=-1)

# Variables
labels = list(range(len(class_colors))) 
colors = list(class_colors.values())     
cmap = ListedColormap(colors)
norm = BoundaryNorm(labels, cmap.N)

# Create the figure and subplots
num_images = 10 
fig, axs = plt.subplots(num_images, 3, figsize=(12, 4 * num_images))

# Iterate through the image sets
for i in range(num_images):
    index = i * 10  
    if index >= len(X_train):
        break
    # Display the original image
    axs[i, 0].imshow(X_val[index])
    axs[i, 0].set_title('Original Image')
    axs[i, 0].axis('off')
    # Display the original label
    axs[i, 1].imshow(y_val_np[index], cmap=cmap, vmin=0, vmax=8)
    axs[i, 1].set_title('Original Label')
    axs[i, 1].axis('off')
    # Display the predicted label
    axs[i, 2].imshow(y_pred_np[index], cmap=cmap, vmin=0, vmax=8)
    axs[i, 2].set_title('Predicted Label')
    axs[i, 2].axis('off')

# Display the figure and adjust the spacing between subplots
plt.subplots_adjust(wspace=0.2, hspace=0.2)
plt.show()